## MediaPipe

- holistic 객체의 
- face_landmarks = 얼굴 랜드마크
- left_hand_landmarks = 왼쪽 손 랜드마크
- right_hand_landmarks = 오른쪽 손 랜드마크
- pose_landmarks = pose 랜드마크
- [mediapipe-documentation]https://google.github.io/mediapipe/getting_started/python.html



 ## 1. Install dependencies

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import mediapie_keypotint as mk
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard # 로깅
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

: 

## 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing.mk.draw_landmarks

<function mediapipe.python.solutions.drawing_utils.draw_landmarks(image: numpy.ndarray, landmark_list: mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList, connections: Union[List[Tuple[int, int]], NoneType] = None, landmark_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[int, mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2), connection_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[Tuple[int, int], mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2))>

In [ ]:
# 영상 테스트
file_num = '0'
ear_format = f'train_data/ear/{file_num}.avi'

In [ ]:
'''
    비디오 프레임 측정 
'''
cap = cv2.VideoCapture(ear_format)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)

In [ ]:
# 웹캠을 실행시켜 landmark를 감지하는 블록.

cap = cv2.VideoCapture(ear_format)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Frame 길이 측정. 
print(f'length : {length}')
frame_jump = round(length / 30)
print(frame_jump)

frame_cnt = 0 
# frame 측정.
saved_frames = 0 
prev = None
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
    
        # frame을 읽는다.
        ret, frame = cap.read()

        # 읽어올 거 없으면 알아서 끄세요.
        if not ret:
            while saved_frames < 30:
                print(f"new frame {saved_frames} added")
                saved_frames += 1
            break
        
        frame_cnt += 1
        prev = frame
        
        if saved_frames < 30 and frame_cnt % frame_jump == 0:
            print(f"saved {saved_frames}")
            saved_frames += 1

            # Make detections   
            image, results = mediapipe_detection(frame, holistic)
        
            # Draw landmarks
            draw_styled_landmarks(image, results)

            # video show
            cv2.imshow("WebCam", image)

        # 끄고싶을때 사용하는 키
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
        
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
len(results.face_landmarks.landmark)

In [ ]:
results

In [ ]:
draw_landmarks(prev, results)

In [ ]:
prev.shape

In [ ]:
plt.imshow(cv2.cvtColor(prev, cv2.COLOR_BGR2RGB))

## 3. Extract Keypoint Values

In [ ]:
results.pose_landmarks.landmark[0].visibility

In [7]:
pose = []
for res in results.pose_landmarks.landmark:
    # randmark의 좌표 추출
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

NameError: name 'results' is not defined

In [8]:
# pose의 landmark 배열을 일차원으로 펴서 반환한다. pose landmark가 없을 시 동일한 shape를 가진 영행렬을 반환한다.
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

# face의 landmark 배열을 일차원으로 펴서 반환. face landmark가 없을 시 동일한 shape를 가진 영행렬을 반환한다.
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

# 왼손 랜드마크 배열 left_hand_landmarks가 없을 경우에는 동일한 shape의 영행렬을 반환한다.
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

# 오른손 랜드마크 배열 left_hand_landmarks가 없을 경우에는 동일한 shape의 영행렬을 반환한다.
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

NameError: name 'results' is not defined

In [ ]:
print(f'right hand shape : {rh.shape}')
print(f'lefh hand shape : {lh.shape}')
print(f'pose shape : {pose.shape}')
print(f'face shape : {face.shape}')
print(f'all mediapipe shape : {126 + 132 + 1404}')

In [ ]:
# face의 landmark에서 x, y, z의 총수를 곱한 것이 face_lanemark를 1차원 행렬로 폈을때의 shape와 같다.
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() \
if results.face_landmarks \
else np.zeros(1404)


In [10]:
# skelleton의 keypoints 추출
mk.extract_keypoints(results).shape

NameError: name 'results' is not defined

In [ ]:
extract_keypoints(results)[:-10]

## 4. Setup Folders for Collection

In [8]:
actions = os.listdir("train_data/")

In [9]:
# 추출할 numpy array 타입의 데이터 PATH
DATA_PATH = os.path.join("Multi_Class_Data")

# 특정 행동 들을 감지하려는 작업 (hello, thanks, iloveyou)
actions = np.array(actions)

# 비디오 숫자.
no_sequences = 20

# 비디오 내의 전체 프레임
sequence_length = 30

In [ ]:
# hello
## 0
## 1
## 2
## ...
## 29

# thanks

# I love you

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass


In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [ ]:
# 각프레임 데이터를 numpy 데이터 형태로 저장.
'''
현재 hello 단어의 프레임이 0부터 ~ 29 까지 인데
예시로 hello의 각 프레임마다 numpy 형태로 데이터가 저장되어서
해당 단어를 인식한다.    
'''
np.save('0', result_test)

In [29]:
# 저장한 numpy 형태의 배열 파일을 불러온다.
np_result = np.load('0.npy')

In [31]:
np_result

array([ 0.51253808,  0.22885615, -0.83650005, ...,  0.        ,
        0.        ,  0.        ])

In [30]:
np_result.shape

(1662,)

## 5. Collect Keypoint Sequences

In [ ]:
# 영상 테스트
file_num = '0'
ear_format = f'train_data/ear/{file_num}.avi'

In [10]:
TRAIN_DATA_PATH = os.path.join("train_data")

In [11]:
word_folder = os.listdir(TRAIN_DATA_PATH)

In [ ]:
video_file = dict()

In [ ]:
for word in word_folder:
    for action in actions:
        array = []
        test = os.path.join(TRAIN_DATA_PATH,word)
        test_file = os.listdir(test)
        for t in test_file:
            test_x = os.path.join(test,t)
            array.append(test_x)
        video_file[action] = array
        
        

In [ ]:
video_file

In [ ]:
video_file

In [ ]:
# 단어 행동 데이터를 수집해서 각 시퀀스를 numpy 배열 형태로 저장한다.

# Loop through actions ex) ['ear', 'snow', 'leg']
for action in actions:
    
    for sequence in range(no_sequences): # 20개 루프
        cap = cv2.VideoCapture(video_file[action][sequence])
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Frame 길이 측정. 
        frame_jump = round(length / 30)

        frame_cnt = 0 
        # frame 측정.
        saved_frames = 0 # (saved_frames).npy 파일로 저장.
        prev = None

        # Set mediapipe model
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

            # Loop through sequences aka videos ex) no_sequences == 20 비디오 개수
                
            print(f'sequence : {sequence}')
            # Loop through video length aka sequence length
            '''
                해야 되는 것
                원래 video frame을 잘라서. 
                30 프레임 형태로 만들어줘야함.
            '''
            while cap.isOpened():
                
                # frame을 읽는다.
                ret, frame = cap.read()

                
                # 읽어올 거 없으면 알아서 끄세요.
                if not ret:
                    break
                
                frame_cnt += 1
                
                # 4초 아니면 3초 아니면 2초 당 frame을 추출하는 로직. frame_jump가 영상 길이.
                if saved_frames < 30 and frame_cnt % frame_jump == 0:

                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)

                    prev = results

                    # Draw landmarks
                    draw_styled_landmarks(image, results)

                    '''
                        내보낼 keypoints 들
                        1. 각 프레임 마다 keypoints를 추출한다.
                        2. 각 프레임 마다 DATA_PATH에 맞게 해당 keypoints를 numpy 데이터 형태로 npy_path에 저장한다.
                        3. 해당 numpy 데이터 형태를 저장한다.
                    '''
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(saved_frames))

                    print(f'npy_path : {npy_path}')

                    np.save(npy_path, keypoints)

                    print(f"saved {saved_frames}")
                    saved_frames += 1
                    
                    # 스크린에 보여준다.
                    cv2.imshow('OpenCV Feed', image)
                    
                # # 끄고싶을때 사용하는 키
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
            while saved_frames < 30:
                keypoints = extract_keypoints(prev)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(saved_frames))
                print(f"new frame {saved_frames} added")
                np.save(npy_path, keypoints)
                saved_frames += 1

        cap.release()
        cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

## 6. Preprocess Data and Create Labels and Features

In [12]:
# train, test 데이터 분할


In [13]:
# actions 단어들에 labeling
label_map = {label:num for num, label in enumerate(actions)}

In [14]:
'''
단어마다 30개의 프레임이 존재.
단어 * 30 개의 넘파이 배열에서
keypoints를 나타내는 총 1662개의 특정 값들이 필요함.
'''
label_map

{'back': 0,
 'belly': 1,
 'bleeding': 2,
 'bridge': 3,
 'burn': 4,
 'choke': 5,
 'colic': 6,
 'cut': 7,
 'dislocation': 8,
 'ear': 9,
 'faint': 10,
 'fever': 11,
 'Fracture': 12,
 'hand': 13,
 'head': 14,
 'heart_attack': 15,
 'high_fever': 16,
 'inside_of_thigh': 17,
 'knee': 18,
 'left_eye': 19,
 'nose': 20,
 'painkiller': 21,
 'right_eye': 22,
 'seizure': 23,
 'shortness of breath': 24,
 'shoulder': 25,
 'snow': 26,
 'whirl': 27,
 'wrist': 28}

In [18]:
# sequences, labels 배열
sequences, labels = [], [] 

for action in actions:
    # 각 프레임 마다.(이 예제에서는 30)
    for sequence in range(no_sequences):
        window = []

        # sequence의 길이마다(이 예제에서는 30) 
        for frame_num in range(sequence_length):

            '''
            각 sequence의 numpy 형태 배열을 불러온다.
            ex) hello > 0 > 0.npy
            window.append(hello > 0 > 0.npy)
            '''
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # 해당 numpy 배열을 window 배열에 추가한다.
            window.append(res)

        '''
            각 frame의 배열 모음을 추가한다.
            ex) hello > 0(numpy 배열 모음 0 ~ 30.npy)

            frame 배열에 라벨링을 추가해준다.
            ex) sequences = [
                [hello.0], [hello.1], [hello.2] ....
                [thanks.0], [thanks.1], [thanks.2] ...
            ]
            
            labels_map = { 'hello': 0, 'thanks': 1, 'iloveyou': 2}
            labels =[
                0, 0, 0, 0, 0 ....
                1, 1, 1, 1, 1 ....
                2, 2, 2, 2, 2 ....
            ]
        '''
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
'''
1. sequences의 배열에는 각 단어마다 30개의 sequence
총 90개의 sequence를 가진다.

2. 각 시퀀스(프레임)마다 30번의 keypoints 추출 작업을 거친다.
3. 총 1662개의 key 포인트를 가지는 numpy 배열을 가진다.
sequences.shape = (90, 30, 1662)

labels_map = { 'hello': 0, 'thanks': 1, 'iloveyou': 2}
            labels =[
                0, 0, 0, 0, 0 ....
                1, 1, 1, 1, 1 ....
                2, 2, 2, 2, 2 ....
            ]
'''
print(f'sequences의 shape : {np.array(sequences).shape}')
print(f'sequences의 길이 : {len(sequences)}')
print(f'labels의 shape : {np.array(labels).shape}')
print(f'labels의 길이 : {len(labels)}')

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [ ]:
print(f'X_train : {X_train.shape}')
print(f'train_label : {y_train.shape}')
print(f'X_test : {X_test.shape}')
print(f'test_label : {y_test.shape}')

## 7. Build and Train LSTM Neural Network

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
'''
    LSTM Neural network
'''
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) # (frame , keypoints)
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

'''
    One-Hot-Encoding
    ex) res = [0.2, 0.7, 0.1]
    가장 확률 높은 인덱스 번호를 추출 
        => (모델이 예측한 확률이 가장 높은 인덱스 추출.)
        => 예측. (방금한 행동은 아마 1번 일거야)
    np.argmax(res) == 1 
    actions[np.argmax(res)] == 'thanks'
    너가 방금한 행동은 'thanks'야
'''
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
X.shape

In [ ]:
res = [.2, 0.7, 0.1]

In [ ]:
np.argmax(res)

In [ ]:
actions[np.argmax(res)]

In [19]:
'''
다중 클래스 분류 시 일반적인 손실함수 : categorical_crossentropy
다중 클래스 분류 시 일반적인 정확도 측정 함수 : categorical_accuracy
'''
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

In [18]:
'''
    Logs\train 폴더에 들어간뒤
    tensorboard --logdir=. 을 입력하면 
    Tensorbaord화면이 나온다.
    ex) 
    1step. cd suhwa_dataset\Logs\train
    2step. tesorboard --logdir=.
'''

'\n    Logs\train 폴더에 들어간뒤\n    tensorboard --logdir=. 을 입력하면 \n    Tensorbaord화면이 나온다.\n    ex) \n    1step. cd suhwa_dataset\\Logs\train\n    2step. tesorboard --logdir=.\n'

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 29)                957       
Total params: 597,533
Trainable params: 597,533
Non-trainable params: 0
__________________________________________________

## 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
# 예측.
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[0])]

## 9. Save Weights

In [ ]:
# model 저장
model.save('multi_pose.h5')

In [ ]:
del model

In [20]:
# 모델 불러오기 load_model
'''
    1. 모델 재구축 실행 => model build
    2. 모델 컴파일 => model compile
    3. 모델 불러오기 => lodad_weights
'''
model.load_weights('multi_pose.h5')

## 10. Evaluation using Confusion Matrix and Accuracy

In [21]:
'''
    Confusion Matrix를 통한 정확도 검증
'''


In [ ]:
yhat = model.predict(X_train)

In [ ]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue,yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [22]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
file_num = '0177'
snow_format = f'data/snow/KETI_SL_000000{file_num}.avi'

In [ ]:
file_num = "0150"
file_format = f'train_data/ear/1.avi'

In [27]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

for i in range(20):
    cap = cv2.VideoCapture(f'train_data/ear/{i}.avi')
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            if not ret:
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)
            
            # Draw landmarks
            draw_styled_landmarks(image, results)
            
            # 2. Prediction logic
            keypoints = extract_keypoints(results)
    #         sequence.insert(0,keypoints)
    #         sequence = sequence[:30]
            sequence.append(keypoints)
            sequence = sequence[-30:]
            
            if len(sequence) == 30:
                # numpy 배열 차원 추가
                # 이 예제에서는 (30, 1662) => (1, 30, 1662)
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                
                
                #3. Viz logic
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
                image = prob_viz(res, actions, image, colors)
                
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

IndexError: list index out of range

In [25]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.03

for i in range(20):
    cap = cv2.VideoCapture(f'train_data/ear/{i}.avi')
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            if not ret:
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)
            
            # Draw landmarks
            draw_styled_landmarks(image, results)
            
            # 2. Prediction logic
            keypoints = extract_keypoints(results)
    #         sequence.insert(0,keypoints)
    #         sequence = sequence[:30]
            sequence.append(keypoints)
            sequence = sequence[-30:]
            
            if len(sequence) == 30:
                # numpy 배열 차원 추가
                # 이 예제에서는 (30, 1662) => (1, 30, 1662)
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(f'predict result : {actions[np.argmax(res)]}')
                print(f'res : {res}')
                
                
                #3. Viz logic
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
                # image = prob_viz(res, actions, image, colors) # color를 class 개수만큼 준비해야 함.
                
            # cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            # cv2.putText(image, ' '.join(sentence), (3,30), 
            #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Show to screen
            # cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [28]:
print(f'predict result : {actions[np.argmax(res)]}')

predict result : hand


In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
# 모델의 예측 규격에 맞게 데이터를 캡슐화
np.expand_dims(X_test[0].shape, axis=0)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))